# Entendendo essa belezinha

In [3]:
import pandas as pd

In [4]:
path = r'C:\Users\User\Documents\Estudo\DA\AlphaClub\data\alpha_club_database.xlsx'

clientes = pd.read_excel(path, sheet_name='Clientes')
vendas = pd.read_excel(path, sheet_name='Vendas')
entradas = pd.read_excel(path, sheet_name='Entradas')
restaurante = pd.read_excel(path, sheet_name='Restaurante')

In [10]:
# Vou agrupar os dataframes para fazer uma análise inicial dos dados problemáticos.

dfs = {
    "clientes": clientes,
    "vendas": vendas,
    "entradas": entradas,
    "restaurante": restaurante
    }

In [14]:
# Valores Nulos

for nome, df in dfs.items():
    print(nome, df.isna().sum().sum())



clientes 29208
vendas 0
entradas 0
restaurante 0


In [24]:
# Tipos

for nome, df in dfs.items():
    print(f'\n{nome}\n')
    print(df.dtypes)


clientes

ID                    int64
Nome                 object
Idade                 int64
Regiao               object
Contrato             object
Carteirinha          object
Inspecao_Medica      object
Primeiro_Contrato    object
Ultimo_Contrato      object
Recomendacao         object
dtype: object

vendas

ID_Vendedor     int64
Data           object
ID_Cliente      int64
Cliente        object
Contrato       object
Vendedor       object
dtype: object

entradas

ID                int64
Data             object
Hora_Entrada     object
Hora_Saida       object
ID_Cliente        int64
Cliente          object
Acompanhantes     int64
Visitante        object
dtype: object

restaurante

Data            object
ID_Cliente       int64
Cliente         object
Almoco         float64
Bebidas         object
Total_Gasto    float64
dtype: object


In [22]:
# Verificando duplicatas

for nome, df in dfs.items():
    print(nome, df.duplicated().sum())

clientes 0
vendas 0
entradas 0
restaurante 0


In [23]:
# Verificando a tipagem das colunas

for nome, df in dfs.items():
    print(f"\n{nome}")
    print(df.dtypes)


clientes
ID                    int64
Nome                 object
Idade                 int64
Regiao               object
Contrato             object
Carteirinha          object
Inspecao_Medica      object
Primeiro_Contrato    object
Ultimo_Contrato      object
Recomendacao         object
dtype: object

vendas
ID_Vendedor     int64
Data           object
ID_Cliente      int64
Cliente        object
Contrato       object
Vendedor       object
dtype: object

entradas
ID                int64
Data             object
Hora_Entrada     object
Hora_Saida       object
ID_Cliente        int64
Cliente          object
Acompanhantes     int64
Visitante        object
dtype: object

restaurante
Data            object
ID_Cliente       int64
Cliente         object
Almoco         float64
Bebidas         object
Total_Gasto    float64
dtype: object


As datas estão como object. O que é comum pegando um arquivo xlsx

In [25]:
# Vendo se há anormalidades nas quantidades dos itens categóricos

for nome, df in dfs.items():
    print(f"\n{nome}")
    for col in df.select_dtypes(include="object"):
        print(col, "->", df[col].nunique())


clientes
Nome -> 2499
Regiao -> 3
Contrato -> 6401
Carteirinha -> 2
Inspecao_Medica -> 2
Primeiro_Contrato -> 4572
Ultimo_Contrato -> 5419
Recomendacao -> 2

vendas
Data -> 4572
Cliente -> 50
Contrato -> 3
Vendedor -> 19

entradas
Data -> 7192
Hora_Entrada -> 420
Hora_Saida -> 780
Cliente -> 2494
Visitante -> 2

restaurante
Data -> 6996
Cliente -> 2491
Bebidas -> 40


#                Primeira Análise

## Verificação de valores nulos

Foi realizada a contagem de valores ausentes em cada base de dados.

- clientes: 29.208 valores nulos
- vendas: 0
- entradas: 0
- restaurante: 0

Observa-se que apenas a base **clientes** apresenta dados faltantes.

In [17]:
clientes.isna().sum()

ID                       0
Nome                     0
Idade                    0
Regiao                   0
Contrato                 0
Carteirinha              0
Inspecao_Medica          0
Primeiro_Contrato    14604
Ultimo_Contrato      14604
Recomendacao             0
dtype: int64

In [1]:
clientes_nulos = clientes[clientes['Primeiro_Contrato'].isna()]
clientes_nulos.head(3)

NameError: name 'clientes' is not defined

## Verificação de valores nulos

A presença de valores nulos não está associada a erros de coleta, mas à própria regra de negócio da base.
As colunas de primeiro e último contrato são preenchidas apenas para clientes que já encerraram ou possuem histórico de contratos. O DataFrame de clientes inclui **Dependentes**, que não necessariamente possuem contratos próprios.

Dessa forma, os valores ausentes são esperados e necessários, e serão tratados considerando esse contexto durante as análises.


## Padronização de textos

Foi avaliada a possibilidade de inconsistências em colunas categóricas (espaços extras ou variações de escrita).
A análise de valores únicos não indicou categorias duplicadas ou ruído textual, portanto nenhuma padronização adicional foi necessária nesta etapa.


In [37]:
# Criando a coluna "Gestão" classificando o ano da venda para utilizar na analise.

vendas["Gestão"] = (
    vendas["Data"].dt.year
    .apply(lambda x: "Marrone" if x <= 2012
           else "Corleone" if x <= 2022
           else "Saad")
)


In [39]:
# Gerando os csvs tratados

import os

pasta = "../data/processed"
os.makedirs(pasta, exist_ok=True)

for nome, df in dfs.items():
    df.to_csv(f"{pasta}/{nome}_tratado.csv", index=False)